In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

raw_df = pd.read_csv('/Users/agnespark/fds/data/ll84.csv', na_values='Not Available')
final_df = pd.read_csv('/Users/agnespark/fds/data/final.csv')

In [175]:
raw_df.columns

Index(['Property Id', 'Property Name', 'Parent Property Id',
       'Parent Property Name', 'Month', 'Natural Gas Use  (kBtu)',
       'Electricity Use  (kBtu)'],
      dtype='object')

In [46]:
final_df.columns

Index(['Property Id', 'Property Name', 'Address 1', 'Postcode',
       'Primary Property Type - Portfolio Manager-Calculated', 'Year Built',
       'Number of Buildings', 'Occupancy', 'Metered Areas (Energy)',
       'Metered Areas  (Water)', 'ENERGY STAR Score', 'Site EUI (kBtu/ft²)',
       'Weather Normalized Site EUI (kBtu/ft²)',
       'National Median Site EUI (kBtu/ft²)',
       '% Difference from National Median Site EUI', 'Site Energy Use (kBtu)',
       'Weather Normalized Site Energy Use (kBtu)',
       'Weather Normalized Site Electricity Intensity (kWh/ft²)',
       'Weather Normalized Site Natural Gas Intensity (therms/ft²)',
       'Source EUI (kBtu/ft²)', 'Weather Normalized Source EUI (kBtu/ft²)',
       'National Median Source EUI (kBtu/ft²)',
       '% Difference from National Median Source EUI',
       'Source Energy Use - Adjusted to Current Year (kBtu)',
       'Source Energy Use (kBtu)',
       'Weather Normalized Source Energy Use (kBtu)', 'Natural Gas Use (kBtu

-- Remove PIDs that are not in the cleaned/merged dataset.

In [2]:
df1 = raw_df.drop(columns=['Parent Property Id', 'Parent Property Name'])
df1.rename(columns={'Natural Gas Use  (kBtu)': 'gas', 'Electricity Use  (kBtu)': 'electricity', 'Property Id': 'pid', 'Property Name': 'pname', 'Month': 'month'}, inplace=True)

# ONLY USE THESE IDS
df1 = df1[df1['pid'].isin(final_df['Property Id'].unique())]

-- Assess for missing values for electricity and gas usage

In [3]:
df2_elec_mv = df1[df1['electricity'].isna()]
elec_mv_pid_arr = list(df2_elec_mv['pid'].unique())
df2_gas_mv = df1[df1['gas'].isna()]
gas_mv_pid_arr = list(df2_gas_mv['pid'].unique())
missing_both_pids = [x for x in gas_mv_pid_arr if x in elec_mv_pid_arr]

print("# of missing electricity values:", df2_elec_mv.shape[0])
print("# of missing gas values:", df2_gas_mv.shape[0])
print("PIDs with missing electricity values:", len(elec_mv_pid_arr))
print("PIDs with missing gas values:", len(gas_mv_pid_arr))
print("PIDs with missing values for both:", len(missing_both_pids))

# of missing electricity values: 387
# of missing gas values: 28546
PIDs with missing electricity values: 90
PIDs with missing gas values: 3310
PIDs with missing values for both: 51


In [4]:
def array_diff(a, b):
  c = list(set(a) - set(b))
  return c

-- Df of location categories and property type

In [5]:
df3_location_info = final_df[['Property Id', 'Postcode', 'Borough', 'NTA', 'Primary Property Type - Portfolio Manager-Calculated']]
df3_location_info = df3_location_info.rename(columns={'Primary Property Type - Portfolio Manager-Calculated': 'Type'})
df3_location_info.columns

Index(['Property Id', 'Postcode', 'Borough', 'NTA', 'Type'], dtype='object')

In [6]:
df3_elec_mv_locs = df3_location_info[df3_location_info['Property Id'].isin(elec_mv_pid_arr)]
# df3_elec_mv_locs.groupby('Type')['Property Id'].count().sort_values(ascending=False)

In [7]:
df3_gas_mv_locs = df3_location_info[df3_location_info['Property Id'].isin(gas_mv_pid_arr)]
# missing_gas_locations.groupby('Type')['Property Id'].count().sort_values(ascending=False)

-- Df with monthly energy usage and location info

In [10]:
gas_mv_nta_arr = df3_gas_mv_locs['Type'].unique().tolist()
elec_mv_nta_arr = df3_elec_mv_locs['Type'].unique().tolist()

-- Replace missing values with means for property types

In [9]:
df4_join_nta_usage = df1.set_index('pid').join(df3_location_info.set_index('Property Id'))
df4_join_nta_usage['pid'] = df4_join_nta_usage.index
df4_join_nta_usage = df4_join_nta_usage.reset_index(drop=True)
df4_join_nta_usage

,pname,month,gas,electricity,Postcode,Borough,NTA,Type,pid
0,1155,18-Jan,NaN,2191816.3,10036,MANHATTAN,Midtown-Midtown South,Office,7365
1,1155,18-Feb,NaN,1881934.0,10036,MANHATTAN,Midtown-Midtown South,Office,7365
2,1155,18-Mar,NaN,1960262.3,10036,MANHATTAN,Midtown-Midtown South,Office,7365
3,1155,18-Apr,NaN,1842159.3,10036,MANHATTAN,Midtown-Midtown South,Office,7365
4,1155,18-May,NaN,1893434.5,10036,MANHATTAN,Midtown-Midtown South,Office,7365
...,...,...,...,...,...,...,...,...,...
272443,1080 Holdings,18-Aug,NaN,166430.3,11385,QUEENS,Ridgewood,Other,7538772
272444,1080 Holdings,18-Sep,NaN,134362.2,11385,QUEENS,Ridgewood,Other,7538772
272445,1080 Holdings,18-Oct,NaN,100183.2,11385,QUEENS,Ridgewood,Other,7538772
272446,1080 Holdings,18-Nov,NaN,94622.7,11385,QUEENS,Ridgewood,Other,7538772


-- Replace missing values with medians by property type

In [13]:
for ptype in gas_mv_nta_arr:
  med = df4_join_nta_usage[df4_join_nta_usage['Type'] == ptype]['gas'].median()
  df4_join_nta_usage.loc[df4_join_nta_usage['Type'] == ptype] = df4_join_nta_usage.loc[df4_join_nta_usage['Type'] == ptype].fillna({'gas': med})

In [14]:
for ptype in elec_mv_nta_arr:
  med = df4_join_nta_usage[df4_join_nta_usage['Type'] == ptype]['electricity'].median()
  df4_join_nta_usage.loc[df4_join_nta_usage['Type'] == ptype] = df4_join_nta_usage.loc[df4_join_nta_usage['Type'] == ptype].fillna({'electricity': med})

In [15]:
df4_join_nta_usage[df4_join_nta_usage['gas'].isna()]

,pname,month,gas,electricity,Postcode,Borough,NTA,Type,pid


In [16]:
df4_join_nta_usage[df4_join_nta_usage['electricity'].isna()]

,pname,month,gas,electricity,Postcode,Borough,NTA,Type,pid


-- Export as new dataset

In [181]:
out_df = df4_join_nta_usage[['pid', 'pname', 'Type', 'month', 'gas', 'electricity', 'Postcode', 'Borough', 'NTA']]
out_df.rename(columns={'pid': 'PID', 'pname': 'PName', 'month': 'Month', 'gas': 'Gas Usage', 'electricity': 'Electricity Usage'}, inplace=True)
out_df.to_csv('/Users/agnespark/fds/data/monthly_usage.csv', index=False)